## Demonstration of ogcapi API

The initial attempt to use the `owslib` library failed due to issus in the requests getting through to the F5.

Work from here down uses `requests` directly.

In [1]:
import folium
import requests

### Get some GeoJSON data and display it directly on a `folium` map.

In [2]:
# Get the data
params = {'f': 'json', 'limit': 10}
response = requests.get('https://ogcapi.bgs.ac.uk/collections/recentearthquakes/items', params=params)
data = response.json()

In [3]:
# Define an initial map centred on and zoomed to the UK
m = folium.Map(location=[55, -2], zoom_start=5)

In [4]:
# Use a style function to set the style dynamically per data point
# In this case use the magnitude to set the radius of the marker
def style_function(feature):
    radius = feature['properties']['ml'] * 5
    return {'radius': radius,
            'color': 'red',
            'weight': 2}

In [5]:
# Create a basic unstyled tooltip
tooltip = folium.features.GeoJsonTooltip(
    fields=['ml'],
    aliases=['magnitude'])

In [6]:
# Create a basic unstyled popup
popup = folium.features.GeoJsonPopup(
    fields=["year", "latitude", "longitude", "depth", "ml", "intensity"],
    aliases=["year", "latitude", "longitude", "depth", "magnitude", "intensity"])

In [7]:
# Populate and display the map
folium.GeoJson(
    data,
    tooltip=tooltip,
    popup=popup,
    style_function=style_function,
    marker=folium.CircleMarker()
).add_to(m)
m

### Now get the data into geopandas, transform it and then create a map

In [8]:
import geopandas as gpd
import pandas as pd
import branca.colormap as cm
from shapely.geometry import Polygon

In [37]:
# Get all the data in batches of 'limit'.
# This is a bit ugly, there may be a neater way.
earthquake_list = []
offset = 0
limit = 500
params = {'f': 'json', 'offset': offset, 'limit': limit}
response = requests.get('https://ogcapi.bgs.ac.uk/collections/recentearthquakes/items', params=params)
data = response.json()
while len(data['features']) == limit:
    # Get JSON into GeoPandas
    earthquake_list.append(gpd.GeoDataFrame.from_features(data['features']))
    # Get the next batch
    offset += limit
    params = {'f': 'json', 'offset': offset, 'limit': limit}
    response = requests.get('https://ogcapi.bgs.ac.uk/collections/recentearthquakes/items', params=params)
    data = response.json()

# Add the final batch if not empty
if len(data['features']) > 0:
    earthquake_list.append(gpd.GeoDataFrame.from_features(data['features']))

# Concatenate dataframes into one
earthquakes = pd.concat(
    earthquake_list,
    ignore_index=True,
)
# Set geometry
earthquakes = earthquakes.set_geometry('geometry')
earthquakes.tail()

,geometry,earthquake_event_id,datetime,year,latitude,longitude,depth,ml,intensity,user_entered,date_entered,user_updated,date_updated
9916,POINT (-5.69200 57.69600),116388.0,2022-09-27T01:34:11,2022,57.696,-5.692,7.6,1.0,0.0,SYSTEM,2022-10-05T06:00:02,None,None
9917,POINT (-2.86400 52.62700),116389.0,2022-09-27T02:39:08,2022,52.627,-2.864,9.3,1.2,0.0,SYSTEM,2022-10-05T06:00:02,None,None
9918,POINT (-5.72200 57.69200),116390.0,2022-09-27T20:33:37,2022,57.692,-5.722,7.7,1.3,0.0,SYSTEM,2022-10-05T06:00:02,None,None
9919,POINT (-5.71200 57.69700),116391.0,2022-09-28T09:28:41,2022,57.697,-5.712,7.1,1.1,0.0,SYSTEM,2022-10-05T06:00:02,None,None
9920,POINT (-3.88400 56.17500),116392.0,2022-10-03T15:07:51,2022,56.175,-3.884,6.5,1.2,0.0,SYSTEM,2022-10-05T06:00:02,None,None


In [38]:
earthquakes.dtypes

geometry               geometry
earthquake_event_id     float64
datetime                 object
year                     object
latitude                float64
longitude               float64
depth                   float64
ml                      float64
intensity               float64
user_entered             object
date_entered             object
user_updated             object
date_updated             object
dtype: object

In [39]:
# Rename, convert and select columns. Set the CRS.
earthquakes.rename(columns={'earthquake_event_id': 'id', 'ml': 'magnitude'}, inplace=True)
# earthquakes['datetime'] = pd.to_datetime(earthquakes['datetime']) # leave as string for now
earthquakes['id'] = earthquakes['id'].astype(int)
earthquakes = earthquakes[['id', 'datetime', 'magnitude', 'intensity', 'depth', 'geometry']]
earthquakes.set_crs(epsg=4326, inplace=True)
earthquakes.head()

,id,datetime,magnitude,intensity,depth,geometry
0,86123,2017-04-05T15:51:03,1.0,0.0,2.8,POINT (-3.70000 53.86000)
1,92221,1956-05-04T23:26:08,3.0,0.0,NaN,POINT (0.44000 51.88000)
2,92222,1967-11-28T04:07:36,1.8,0.0,NaN,POINT (-3.12000 55.87000)
3,92223,1968-03-08T05:26:48,3.2,0.0,NaN,POINT (-2.97000 55.19000)
4,92224,1968-10-16T17:02:40,3.1,0.0,NaN,POINT (-3.06000 55.93000)


In [40]:
earthquakes.dtypes

id              int64
datetime       object
magnitude     float64
intensity     float64
depth         float64
geometry     geometry
dtype: object

In [41]:
earthquakes.describe()

,id,magnitude,intensity,depth
count,9921.000000,9921.000000,9921.000000,9404.000000
mean,102273.023687,1.697641,0.247858,7.441567
std,8898.718255,0.639596,0.880886,8.124785
min,86123.000000,1.000000,0.000000,0.000000
25%,94836.000000,1.200000,0.000000,1.800000
50%,97413.000000,1.500000,0.000000,5.500000
75%,113129.000000,2.000000,0.000000,10.400000
max,116392.000000,5.400000,6.000000,183.700000


In [42]:
# Get the earthquakes above magnitude 4
big_quakes = earthquakes[earthquakes['magnitude'] >= 4]
big_quakes

,id,datetime,magnitude,intensity,depth,geometry
44,92264,1970-08-09T20:09:01,4.1,5.0,20.9,POINT (-2.47000 54.50000)
52,92272,1971-03-23T20:05:18,4.7,0.0,NaN,POINT (2.80000 59.40000)
113,92336,1974-02-25T20:03:43,4.1,0.0,NaN,POINT (-3.12000 51.64000)
240,92485,1976-08-18T20:45:52,4.2,0.0,NaN,POINT (2.00000 62.00000)
289,92536,1977-04-06T19:32:04,4.4,0.0,NaN,POINT (3.00000 61.50000)
...,...,...,...,...,...,...
9582,115690,2022-08-26T01:47:11,4.2,0.0,19.1,POINT (4.12900 61.80400)
9646,115777,2000-09-23T04:23:45,4.2,5.0,15.2,POINT (-1.61100 52.28200)
9655,115786,2000-12-08T05:54:01,4.5,0.0,9.8,POINT (1.92700 59.94300)
9688,115825,2001-05-07T09:43:33,4.2,0.0,1.0,POINT (3.25900 56.68200)


In [43]:
# Use shapely to get the centre of the map
bounds = Polygon(list(big_quakes['geometry'])).bounds
bounds

(-10.904, 49.153, 5.0, 62.67)

In [44]:
mag_min = big_quakes['magnitude'].min()
mag_max = big_quakes['magnitude'].max()

In [45]:
# Create a colormap using the min and max magnitudes
colormap = cm.LinearColormap(colors=['yellow','red'], caption='Magnitude', vmin=mag_min, vmax=mag_max)

# Define a style function to apply that map
def map_style_function(feature):
    color = colormap(feature['properties']['magnitude'])
    return {'radius': 3,
            'color': color,
            'fill': True,
            'fill_opacity': 1}

# Define an initial map centred on the points
m2 = folium.Map(
    location=[(bounds[1] + bounds[3])/2, (bounds[0] + bounds[2])/2],
    zoom_start=4)

# Create popup
popup = folium.features.GeoJsonPopup(fields=["datetime", "magnitude", "intensity", "depth"])

# Populate the map, converting the dataframe to JSON
folium.GeoJson(
    big_quakes.to_json(),
    popup=popup,
    style_function=map_style_function,
    marker=folium.CircleMarker()
).add_to(m2)

# Add colormap to map and display the map
m2.add_child(colormap)
m2